<a href="https://colab.research.google.com/github/bryanbayup/petpoint/blob/main/PetpointWithGeneralConver.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bryanbayup/petpoint

Cloning into 'petpoint'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 45 (delta 5), reused 29 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 59.60 KiB | 2.48 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
!pip install torch torchvision torchaudio

In [4]:
import pandas as pd

# Load normalization dictionary
normalization_dict = {}
with open('/content/petpoint/normalization/normalization.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # Check if the line contains the delimiter before splitting
        if '\t' in line:
            slang, normal = line.strip().split('\t')
            normalization_dict[slang] = normal
        # Handle lines without the delimiter (e.g., print a warning)
        else:
            print(f"Warning: Skipping line '{line.strip()}', no tab delimiter found.")

# Function to normalize text
def normalize_text(text):
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

In [5]:
# Load stopwords
with open('/content/petpoint/normalization/stopword.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)


In [27]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 24.5 MB/s eta 0:00:00


In [28]:
import stanza
stanza.download('id')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: id (Indonesian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/id/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [29]:
import stanza

# Initialize the Indonesian pipeline
nlp = stanza.Pipeline('id')

def tokenize_and_lemmatize(text):
    doc = nlp(text)
    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words]
    return ' '.join(lemmas)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could 

In [31]:
import json
import glob

intents = []

# Load all JSON files from corpus/id/
for file in glob.glob('/content/petpoint/corpus/id/*.json'):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        intents.extend(data)

# Load domain-specific data
with open('/content/petpoint/kucing_anjing/kucing_anjing.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    intents.extend(data)

In [32]:
import pandas as pd

training_data = []

for intent in intents:
    intent_name = intent['intent']
    for utterance in intent['utterances']:
        # Normalize and preprocess the utterance
        utterance = normalize_text(utterance.lower())
        utterance = remove_stopwords(utterance)
        utterance = tokenize_and_lemmatize(utterance)
        training_data.append({'utterance': utterance, 'intent': intent_name})

df = pd.DataFrame(training_data)

In [33]:
df.head()

,utterance,intent
0,,agent.acquaintance
1,,agent.acquaintance
2,pribadi,agent.acquaintance
3,gambar,agent.acquaintance
4,cerita,agent.acquaintance


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['utterance'])

In [35]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['intent'])

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [40]:
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train, y_train)

LinearSVC()

In [42]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

# Get unique labels in y_test and y_pred
unique_labels = sorted(set(y_test) | set(y_pred))

# Filter le.classes_ to keep only the labels present in y_test and y_pred
target_names = [le.classes_[i] for i in unique_labels]

print(classification_report(y_test, y_pred, target_names=target_names))

                              precision    recall  f1-score   support

          agent.acquaintance       0.11      1.00      0.20         1
                   agent.age       0.00      0.00      0.00         1
             agent.beautiful       0.00      0.00      0.00         0
              agent.beclever       0.00      0.00      0.00         2
                agent.boring       0.00      0.00      0.00         0
                  agent.boss       1.00      1.00      1.00         1
            agent.canyouhelp       0.50      1.00      0.67         1
               agent.chatbot       1.00      1.00      1.00         1
                agent.clever       0.00      0.00      0.00         1
                  agent.fire       1.00      0.50      0.67         2
                 agent.funny       1.00      1.00      1.00         2
                  agent.good       0.00      0.00      0.00         1
                 agent.happy       1.00      1.00      1.00         1
                 ag

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

In [43]:
def predict_intent(text):
    text = normalize_text(text.lower())
    text = remove_stopwords(text)
    text = tokenize_and_lemmatize(text)
    X = vectorizer.transform([text])
    intent_index = model.predict(X)[0]
    intent = le.inverse_transform([intent_index])[0]
    return intent

In [44]:
import random

def get_response(intent_name):
    for intent in intents:
        if intent['intent'] == intent_name:
            return random.choice(intent['answers'])
    return "Maaf, saya tidak mengerti."

In [46]:
sentiment_scores = {}

# Load positive sentiment words
with open('/content/petpoint/sentiment/tsv/positive.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        word, score = line.strip().split('\t')
        sentiment_scores[word] = int(score)

# Load negative sentiment words
with open('/content/petpoint/sentiment/tsv/negative.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        word, score = line.strip().split('\t')
        sentiment_scores[word] = int(score)

In [47]:
def analyze_sentiment(text):
    words = text.split()
    score = 0
    for word in words:
        score += sentiment_scores.get(word, 0)
    return score

In [48]:
def get_response(intent_name, sentiment_score):
    for intent in intents:
        if intent['intent'] == intent_name:
            response = random.choice(intent['answers'])
            if sentiment_score < 0:
                response = "Saya mengerti perasaan Anda. " + response
            elif sentiment_score > 0:
                response = "Senang mendengarnya! " + response
            return response
    return "Maaf, saya tidak mengerti."

In [ ]:
def chatbot_response(user_input):
    intent = predict_intent(user_input)
    sentiment_score = analyze_sentiment(user_input)
    response = get_response(intent, sentiment_score)
    return response

# Chat loop
print("Selamat datang di Chatbot! Ketik 'exit' untuk keluar.")
while True:
    user_input = input("Anda: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Sampai jumpa!")
        break
    bot_response = chatbot_response(user_input)
    print(f"Chatbot: {bot_response}")

Selamat datang di Chatbot! Ketik 'exit' untuk keluar.
Anda: halo
Chatbot: Hai!
Anda: siapa kamu
Chatbot: Anggap saya sebagai agen virtual
Anda: anjing saya sakit
Chatbot: Saya mengerti perasaan Anda. Potong kuku anjing setiap 3-4 minggu, atau ketika kuku mulai terdengar saat anjing berjalan di lantai keras.
Anda: halo
Chatbot: Halo!
Anda: anjing saya muntah
Chatbot: Periksa pola makan dan jenis makanan kucing. Jika muntah berlanjut, konsultasikan dengan dokter hewan.
Anda: kucing saya bulunya rontok
Chatbot: Periksa apakah ada parasit atau alergi. Jika tidak, mungkin kucing mengalami stres. Konsultasikan dengan dokter hewan.
Anda: maksudnya
Chatbot: Aku selalu senang berbicara denganmu, teman
Anda: apa penyebab bulu kucing rontok
Chatbot: Periksa apakah ada parasit atau alergi. Jika tidak, mungkin kucing mengalami stres. Konsultasikan dengan dokter hewan.
